In [1]:
import json
import re
import pandas as pd
import numpy as np
import os

In [35]:
testcaseDir = 'annotations/'
testOutput = 'testanno/'
json_files = [pos_json for pos_json in os.listdir(testcaseDir) if pos_json.endswith('.json')]

# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(testcaseDir, js)) as json_file:
        #print(json_file)
        s = json_file.read()
        s = s.replace('\'','\"')
        data = json.loads(s)
        #data = json.load(json_file)
    
    fileName = (os.path.splitext(js)[0])
    stack=[]
    c=0
    accessJSON(data) #preprocess the datatype to required format
    k,v=seperatekeyvalues(stack)
    rows,columns=findingrowscolumns(k)


    fileDir = os.path.join(testOutput, fileName+".csv")
 #   print(fileDir,rows,columns)
   # print(k, v)
    writingintoFile(fileDir, rows, columns, k, v)   #writes datatype modified column to file

    flag=0
#checking for language tag if any as another json object
    stackLG=[]
    c=0
    accessJSONLT(data)
    if(stackLG):
        k,v=seperatekeyvalues(stackLG)
        rows,columns=findingrowscolumns(k)
        fileDir1 = os.path.join(testOutput, fileName+"_1.csv")
        writingintoFile(fileDir1, rows, columns, k, v)   #writes language tags to another file
        flag=1    

    if(flag==1):
        finalDF=mergeDataset(fileDir,fileDir1)
        os.remove(fileDir1)
    else:
        finalDF = pd.read_csv(fileDir)

    finalD = changeLanguagetag(finalDF)

    if os.path.exists(fileDir):
        os.remove(fileDir)
    finalD.to_csv(fileDir, index=False)    

RMLTC0010c


In [2]:
#accessing nested
#1st JSON object
def accessJSON(data):
    global c
    for key,value in data.items():
       # print(value)
        if(type(value)==type(str())  or type(value)==type(int()) or type(value)==type(float()) or type(value)==type(bool()) ):
            stack.append(key)
            stack.append(value)
            if(key.lower()=="dt"):
                i=stack.pop()
                #print(stack)
                if(i.lower()=="int" or i.lower() == "dec" or i.lower() =="str" or i.lower()=="bool" or i.lower() == "integer" or i.lower() =="decimal"  or i.lower() =="string" or i.lower()=="boolean"):
                    if(i.lower()=="int" or i.lower()=="integer"):
                        i="xsd:integer"
                    elif(i.lower()=="dec" or i.lower() == "decimal"):
                        i="xsd:decimal"
                    elif(i.lower()=="bool" or i.lower() == "boolean"):
                        i="xsd:boolean"
                    else:
                        i="xsd:string"
                elif(re.search(urlRE,i.lower())):
                        res = re.findall(r'#',i) 
                        i="xsd:"+i[(i.index('#')+1):]
                elif(value==""):
                    j=stack.pop()
                    #print("j",j)
                    k=stack.pop()

                    #print("k",k)
                    stack.append(k)
                    stack.append(j)

                    if(type(k)==type(int())):
                        i="xsd:integer"
                    elif(type(k)==type(bool())):
                        i="xsd:boolean"
                    elif(type(k)==type(float())) :
                        i="xsd:decimal"
                    else:
                        i="xsd:string"
                stack.append(i)

        if type(value) == type(dict()):
            accessJSON(value)
        elif type(value) == type(list()):
            c=c+1
            if(c==2):
                break
            for val in value:
                if type(val) == type(str()):
                    pass
                elif type(val) == type(list()):
                    pass
                else:
                    accessJSON(val)

In [3]:
def findingrowscolumns(keys):
    start=''
    ei=0
    count = 0
    #print("keysLen",len(keys))
    for i in range(len(keys)):
        if(start==''):
            start=keys[i]
            si=0
        elif(keys[i]==start):
            end=keys[i-1]
            ei=i
            break 
        else:
            ei=i 

    j=ei+1

    count = 1
    for i in range(len(keys)):
        if(j >= len(keys)):
            break
        #print("range(EI)",range(ei))
        for k in range(ei):
            #print(keys[k])
            #print(keys[k+ei])
            if(comparekeys(keys[k],keys[k+ei])==False):
                print("Keys are not of same length")
        j=j+ei
        count=count+1
        #print("read ",count,"rows")

    if(j==ei+1):
        return count, ei+1
    elif(ei==0):
        return 0,0
    else:
        return count, ei

In [6]:
#url regular expression
urlRE = re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")

#decRE = re.compile(r"[+-]?([0-9]*[.])?[0-9]+")

In [7]:
def seperatekeyvalues(stackelements):
    keys = []
    values = []

    for i in range(len(stackelements)):
        if(i%2==0):
            keys.append(stackelements[i])
        else:
            values.append(stackelements[i])
    return keys, values

In [8]:
def comparekeys(w1,w2):
    if(w1==w2):
        return True
    else:
        return False

In [9]:
def writingintoFile(fileName, rows, column,keys, values):
    a=np.array(values).reshape(rows, column)
    # convert array into dataframe 
    DF = pd.DataFrame(a, columns=keys[0:column]) 
    # save the dataframe as a csv file 
    DF.to_csv(fileName,index = False)

In [10]:
def accessJSONLT(data):
    global c
    for key,value in data.items():       
        if(type(value) == type(str()) and c==2):
            stackLG.append(key)
            stackLG.append(value)
        if type(value) == type(dict()):
            accessJSONLT(value)
        elif type(value) == type(list()):
            c=c+1
            if(c == 2):
                if(key.lower()!="languages"):
                    print("Error! Required language tag here!")
            for val in value:
                if type(val) == type(str()):
                    pass
                elif type(val) == type(list()):
                    pass
                else:
                    accessJSONLT(val)


In [11]:
def mergeDataset(f1,f2):
    df1 = pd.read_csv(f1)
    header1=df1.columns.tolist()

    df2 = pd.read_csv(f2)
    header2=df2.columns.tolist()

    count=0
    matchedHeader = []
    nextHeader = []

    for i in header1:
        for j in range(len(header2)):
            if(i==header2[j]):
                matchedHeader.append(i)
                count=count+1
                nextHeader.append(header2[j+1])
            if(count>=2):
                break

    res = df1.merge(df2, how='inner', left_on=[matchedHeader[0]], right_on=[matchedHeader[0]])

    return res

In [12]:
def changeLanguagetag(res):
    lTags = pd.read_csv("ltgs.csv")
    flag=0
    header=res.columns.tolist()
    languagePresent = []

    langColumn = ""
    for i in (header):
      if i=="language":
        langcolumn = "language"
        flag=1
      elif i=="lang":
        langColumn = "lang"
        flag=1
    
    if(flag==0):
      #print("returning")
      return res
    
    temp=0
    langPresent = res[langColumn]
    #print(newCol)
    #languagePresent.append(res[[langColumn]])

  #  print(len(res))
    for i in range(len(langPresent)):
        for j in range(len(lTags)):
            if(lTags.loc[j,"Language"].lower() == str(langPresent[i]).lower()):
              langPresent[i] = lTags.loc[j,"tag"]
    
    res[langColumn]=langPresent

    return res
